In [1]:
import vertexai 
from vertexai.generative_models import GenerativeModel, Part, SafetySetting, GenerationResponse
from vertexai.preview.generative_models import Tool, grounding
import json

In [2]:
 # Initialize Vertex AI with project settings
vertexai.init(project="cloud-learning-443407", location="us-central1")

In [262]:
def process_image(uploaded_file):
    if uploaded_file is not None:
        bytes_data = uploaded_file
        return Part.from_data(
            mime_type="image/*",
            data=bytes_data
        )
    return None

In [530]:
def analysis(image_parts, system_instructions, user_prompt, extended_context):

    # Set up safety filters
    analysis_safety_settings = [
        SafetySetting(
            category=cat, threshold=SafetySetting.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE)
        for cat in [
            SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        ]
    ]

    # Configure generation parameters
    analysis_generation_config = {
        "max_output_tokens": 8192,  # Maximum length of generated response
        "temperature": 0.2,  # Controls randomness (0=deterministic, 1=random)
        "top_p": 0.95,  # Nucleus sampling parameter
        "response_mime_type": "application/json",  # Specify JSON response format
        "response_schema": {
            "type": "OBJECT", 
            "properties": {
                "green_box_description": {"type": "STRING"},
                "main_query": {"type": "STRING"},
                "nitty_gritty_details": {"type": "STRING"},
                "web_query": {"type": "STRING"},
                "image_query": {"type": "STRING"},
                "video_query": {"type": "STRING"},
                "subject_categorization": {"type": "STRING"},
                "area_empty": {"type": "STRING"},   
                "score_of_relation": {"type": "STRING"},
                "related_questions": {
                    "type": "ARRAY",
                    "items": {"type": "STRING"}
                },
                "followup_keywords": {  
                    "type": "ARRAY",
                    "items": {"type": "STRING"}
                }
            },
            "required": ["green_box_description", "main_query", "nitty_gritty_details", "web_query", "image_query", "video_query", "subject_categorization", "area_empty", "score_of_relation", "related_questions", "followup_keywords"]
        }
    }

    # Create model instance with system instructions
    model = GenerativeModel(
        "gemini-1.5-flash-002",
        # "gemini-1.5-pro-002",
        #  "gemini-2.0-flash-exp",
        # "gemini-exp-1206",
        # system_instruction=system_instructions
    )

    user_prompt = user_prompt.format(extended_context=extended_context)

    # print(user_prompt)

    # Generate content with streaming enabled
    responses = model.generate_content(
        [user_prompt] + image_parts,
        generation_config=analysis_generation_config,
        safety_settings=analysis_safety_settings,
        stream=False,
    )

    # Accumulate streamed response
    # result = ""
    # for response in responses:
    #     result += response.text
    
    return json.loads(responses.candidates[0].content.parts[0].text)
    # return responses



In [531]:
def explain(analysis, system_instructions, user_prompt):

    # Set up safety filters
    safety_settings = [
        SafetySetting(
            category=cat, threshold=SafetySetting.HarmBlockThreshold.BLOCK_LOW_AND_ABOVE)
        for cat in [
            SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
        ]
    ]

    # Configure generation parameters
    generation_config = {
        "max_output_tokens": 8192,  # Maximum length of generated response
        "temperature": 0,  # Controls randomness (0=deterministic, 1=random)
        "top_p": 0.95,  # Nucleus sampling parameter
        # "response_mime_type": "application/json",  # Specify JSON response format
    #     "response_schema": {"type": "OBJECT", "properties": {"key_topic": {"type": "STRING"},
    #                                                             "full_image_description": {"type": "STRING"},
    #                                                             "cropped_image_description": {"type": "STRING"}}},
    }

    tools = [Tool.from_google_search_retrieval(
        google_search_retrieval=grounding.GoogleSearchRetrieval()
    )]


    # Create model instance with system instructions
    model = GenerativeModel(
        "gemini-1.5-flash-002",
        # "gemini-1.5-pro-002",
        #  "gemini-2.0-flash-exp",
        # "gemini-exp-1206",
        system_instruction=system_instructions,
        tools=tools
    )

    

    # Generate content with streaming enabled
    responses = model.generate_content(
        [analysis],
        generation_config=generation_config,
        safety_settings=safety_settings,
        stream=False,
    )

    result = GenerationResponse.to_dict(responses)

    # Accumulate streamed response
    # result = ""
    # for response in responses:
    #     result += response.text
    
    return result['candidates'][0]['content']['parts'][0].get('text', '')
    # return responses



In [532]:
ANALYSIS_SYSTEM_PROMPT = """

"""

EXPLAIN_SYSTEM_PROMPT = """
- You are a helpful AI Educational Tutor.
- You are given a query and nitty gritty details of the image.
- When Answering you should use words simple and understandable from educational perspective.
- Make you answer more engaging and interesting.
- Only Answer to Follow up query, if its relevant to Main Query. This is very important.
Background :
- User is trying to more about certain focused part of the image.
- In Step 1, we have extracted the query and nitty gritty details of the user's interested area.
- In Step 2, you have to formulate response explaining the topic incorporating the query and nitty gritty details.
- Never let user know that you are understanding from image and answering
Output Format :
- Use markdown format to format the response.
- Follow the pattern
    - Start with Heading(#), which incorporates the query and nitty gritty details but short and crisp.
    - Short paragraph answering/explaining what user is looking for.
        - Highlight important part of this paragraph with bold and italic. Like answer to user's question, important points, etc.
    - 2-3 sub headings(##), followed by 3-4 bullet points about that subheading.
        - Bullet points should be short and crisp.
    - Finally, a conclusion short crisp paragraph summarizing the response with important points.
"""

In [548]:
ANALYSIS_USER_PROMPT = """
1. Analyze the image.
2. There is green box in the image. I want you describe the green box.
3. Extended Context: {extended_context}
4. I want you describe area inside green box which is user's region of interest. If not clear what is user's region of interest, then get some context from the full image. But try not talk about the full image. If not clear from Green Box, then understand position of box and get some context around the area of green box to always answer right answer relevent to context.
5. Output two things 
 - "Main Query" : Frame a search query describing the area in green box. I want this query to include whats in green box. Understand and reason green box area based on its position of full context. Understand what is in green box area by understanding the context of the image.
    - If there is a text in green box, then just include the text in query.
    - Dont leave out any important information from green box.
    - This query can be longer to incorporate all the information from green box.
 - "Nitty Gritty Details" : nitty gritty in depth details of the green area only majorly. Also include the context of the green box only if necessary.
6. "Video Query" : Frame query for video search. This query should be more like best keywords to get relevant videos of topic in green box. As this gets us videos, frame best query to get relevant videos.
7. "Image Query" : Frame query for image search. This query should be more like best keywords to get relevant images of topic in green box. As this gets us images, frame best query to get relevant images.
8. "Web Query" : Frame query for web search. This query should be more like best keywords to get relevant web pages of topic in green box. As this gets us web pages, frame best query to get relevant web pages.
9. "Subject Catgorization" : Categorize the subject of the image like Math, Physics, Chemistry, Biology, etc.
10. "Area Empty" : Is the area in green box empty? or not useful information? Just answer yes or no.
11. "score_of_relation" : Is Extended Context provided by user related to Main Query? Answer with score value between 0 to 1. answer just decimal score
12. "related_questions" : Frame ~4-5 questions related questions user might ask. These questions should incorporate Extended Context. Along wth question, frame short concise 20-40 word answer to that question seperated by &&&. IMPORTANT : NEVER INCLUDE WORDS LILE "GREEN BOX" IN YOUR RESPONSE.
13. "followup_keywords" : Frame ~4-5 two-three word keywords that user might interested to know next. These keywords should incorporate Extended Context. These keywords should not include Same Extended Context again
"""

EXPLAIN_USER_PROMPT = """

"""

In [555]:
# Load test images
with open("./full2.jpg", "rb") as f:
    image1_bytes = f.read()
# with open("./roi2_1.png", "rb") as f:
#     image2_bytes = f.read()

full_part = process_image(image1_bytes)
# roi_part = process_image(image2_bytes)
# image_parts = [full_part, roi_part]
image_parts = [full_part]
extended_context = "Does uranus have moons?"
extended_context = ""

result = analysis(image_parts, ANALYSIS_SYSTEM_PROMPT, ANALYSIS_USER_PROMPT, extended_context)
print(json.dumps(result, indent=2))


_analysis = f"""
Main Query: {result['main_query']}
Nitty Gritty Details: {result['nitty_gritty_details']}
Follow up query user interested in: {extended_context}
"""




{
  "area_empty": "no",
  "followup_keywords": [
    "Vincent van Gogh",
    "Cafe Terrace",
    "Post-Impressionism",
    "Starry Night",
    "oil on canvas"
  ],
  "green_box_description": "The green box contains a depiction of people sitting at tables outside a cafe at night. The style is impressionistic, with visible brushstrokes and a focus on light and color.",
  "image_query": "Vincent van Gogh Cafe Terrace at Night people sitting at tables",
  "main_query": "Vincent van Gogh's painting of a cafe terrace at night, showing people sitting at tables outside under the starry sky. The painting is done in an impressionistic style with visible brushstrokes.",
  "nitty_gritty_details": "The area shows several people seated at small tables, possibly enjoying drinks or conversation. The scene is set at night, with a warm glow emanating from the cafe and the lights of the city visible in the background. The style is characteristic of Van Gogh's post-impressionism, with visible brushstrokes

In [556]:
print(_analysis)

result = explain(_analysis, EXPLAIN_SYSTEM_PROMPT, EXPLAIN_USER_PROMPT)
print(json.dumps(result, indent=2))

# show formatted text in notebook
from IPython.display import display, Markdown
display(Markdown(result))


Main Query: Vincent van Gogh's painting of a cafe terrace at night, showing people sitting at tables outside under the starry sky. The painting is done in an impressionistic style with visible brushstrokes.
Nitty Gritty Details: The area shows several people seated at small tables, possibly enjoying drinks or conversation. The scene is set at night, with a warm glow emanating from the cafe and the lights of the city visible in the background. The style is characteristic of Van Gogh's post-impressionism, with visible brushstrokes and a vibrant color palette.
Follow up query user interested in: 

"# Vincent van Gogh's \"Caf\u00e9 Terrace at Night\": A Detailed Look\n\nVan Gogh's \"Caf\u00e9 Terrace at Night\" is a captivating depiction of a caf\u00e9 terrace at night, brimming with life and illuminated by the warm glow of gas lamps.  The painting, completed around mid-September 1888 in Arles, France, is *remarkable for its innovative use of color and light*, capturing the vibrant atmosp

# Vincent van Gogh's "Café Terrace at Night": A Detailed Look

Van Gogh's "Café Terrace at Night" is a captivating depiction of a café terrace at night, brimming with life and illuminated by the warm glow of gas lamps.  The painting, completed around mid-September 1888 in Arles, France, is *remarkable for its innovative use of color and light*, capturing the vibrant atmosphere of the evening.  It's not just a simple depiction of a scene; it's a *masterful blend of everyday life and cosmic wonder*, showcasing Van Gogh's unique artistic vision.  The painting is currently housed at the Kröller-Müller Museum in Otterlo, Netherlands.

## The Setting and Atmosphere

*   The scene is set on the Place du Forum in Arles, a historically significant location.
*   The café terrace is bustling with people, suggesting a lively social atmosphere.
*   The warm light from the gas lamps contrasts beautifully with the cool tones of the night sky.
*   The painting's vibrant colors and visible brushstrokes create a sense of movement and energy.

## Artistic Style and Techniques

*   The painting is a prime example of Post-Impressionism, a style that builds upon Impressionism but adds more personal expression and symbolism.
*   Van Gogh's distinctive brushstrokes are clearly visible, adding texture and dynamism to the scene.
*   The use of color is bold and expressive, with contrasting warm and cool tones creating a striking visual effect.
*   The painting is notable for its innovative use of light, capturing the interplay between artificial and natural light sources.

## Symbolism and Interpretation

*   The brightly lit café represents warmth, community, and the vibrancy of urban life.
*   The darker areas of the painting, such as the empty street, suggest solitude and introspection.
*   The starry sky may symbolize Van Gogh's fascination with the cosmos and the divine.
*   The painting is a complex interplay of joy and melancholy, intimacy and vastness, humanity and the cosmos.


In conclusion, Van Gogh's "Café Terrace at Night" is more than just a beautiful painting; it's a powerful expression of the artist's unique vision and a testament to his mastery of color, light, and composition.  The painting's vibrant atmosphere, visible brushstrokes, and symbolic undertones continue to captivate viewers and inspire awe, making it one of Van Gogh's most iconic and enduring works.  The painting's location in Arles, France, and its current home at the Kröller-Müller Museum are also significant aspects of its history and accessibility.


In [210]:
user_prompt = f"""
1. Analyze the image.
2. There is green box in the image. I want you describe the green box.
3. I want you describe area inside green box which is user's region of interest. If not clear what is user's region of interest, then get some context from the full image. But try not talk about the full image. If not clear from Green Box, then understand position of box and get some context around the area of green box to always answer right answer relevent to context.
4. Output two things 
 - "Query" : Frame a search query describing the area in green box. I want this query to include whats in green box. Understand and reason green box area based on its position of whole image.
    - If there is a text in green box, then just include the text in query.
    - Dont leave out any important information from green box.
 - "Nitty Gritty Details" : nitty gritty in depth details of the green area only majorly. Also include the context of the green box only if necessary.
5. "Video Query" : Frame query for video search. This query should be more specific to the area in green box to get relevant videos. As this gets us videos, frame best query to get relevant videos.
6. "Image Query" : Frame query for image search. This query should be more specific to the area in green box to get relevant images. As this gets us images, frame best query to get relevant images.
7. "Web Query" : Frame query for web search. This query should be more specific to the area in green box to get relevant web pages. As this gets us web pages, frame best query to get relevant web pages.
8. "Subject Catgorization" : Categorize the subject of the image like Math, Physics, Chemistry, Biology, etc.
9. "Area Empty" : Is the area in green box empty? or not useful information? Just answer yes or no.
"""

In [216]:
user_prompt = f"""
Analyze the image and provide a detailed description of the image.
Can you tell me what is first image and second image with nitty gritty details?
I want you describe second image which is user's region of interest. If not clear what is user's region of interest, then describe the image as a whole. Never talk about first image.
Also can you frame a query to search for the second image? I want this query to include whats focused in image along with bigger context, like objects in image and what this image looks like
"""

user_prompt = f"""
Analyze the image and provide a detailed description of the image.
There is green box in the image. I want you describe the green box.
I want you describe are inside green box which is user's region of interest. If not clear what is user's region of interest, then get some context from the full image. But try not talk about the full image.
Also can you frame a query to search for the area in green box? I want this query to include whats in image and what it depicts in full image context
 - In this query, you should have two sections one is sharp query and other is talking about nitty gritty details of the green area.
"""

# user_prompt = f"""
# 1. Analyze the image.
# 2. There is green box in the image. I want you describe the green box.
# 3. I want you describe area inside green box which is user's region of interest. If not clear what is user's region of interest, then get some context from the full image. But try not talk about the full image.
# 4. Output two things 
#  - "Query" : Frame a search query describing the area in green box. I want this query to include whats in green box. Also include the context of the green box
#     - If there is a text in green box, then just include the text in query.
#  - "Nitty Gritty Details" : nitty gritty in depth details of the green area only majorly. Also include the context of the green box.
# 5. "Concise Description" : Concise description based on Query and Nitty Gritty Details.
# 6. Is Green box area empty? or not useful information?
# """